In [6]:
import math
from textblob import TextBlob as tb
import newspaper
import os
import random
import pickle
from datetime import datetime
from tqdm import tqdm as tq

In [7]:
import import_ipynb
from prepare import collector
from lexrank_summarizer import lexrank
from news_classes import customizedPaper
from news_classes import paragraph

In [8]:
from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import render_template
from flask import request

In [9]:
import _thread
import time

In [10]:
cwd=os.getcwd()+'/'
news_path='files/news_today.pkl'
summarizer=lexrank(.1,.0001)

In [11]:
with open(news_path, 'rb') as input:
    allnews = pickle.load(input)

In [12]:
def worker():
    while True:
        collector.prepare_news()
        print("News Collected")
        collector.prepare_idf()
        print("Idf Stored")
        with open(news_path, 'rb') as input:
            allnews = pickle.load(input)

In [13]:
'''
try:
   _thread.start_new_thread(worker,())
except:
   print ("Error: unable to start thread")
'''

'\ntry:\n   _thread.start_new_thread(worker,())\nexcept:\n   print ("Error: unable to start thread")\n'

In [14]:
def getSummarizedFromList(para_list):
    summ_list=[]
    for paragraph in para_list:
        if len(paragraph.body) == 0:
            obj={'title':paragraph.title, 'url':paragraph.url, 'text':"Noisy Data", 'date':paragraph.date, 'brand':paragraph.brand}
        else:
            obj={'title':paragraph.title, 'url':paragraph.url, 'text':summarizer.summarize(paragraph.body), 'date':paragraph.date, 'brand':paragraph.brand}

        summ_list.append(obj)    
        
    return summ_list

In [15]:
def getMultiDocSummaryFromList(para_list):
    blob_news=""
    for paragraph in para_list:
        blob_news+=paragraph.body
    
    return summarizer.summarize(blob_news,number_of_summarized_sentences=10)

In [16]:
def searchNews(searched_key, min_count):
    key_words=searched_key.split()
 
    para_list=[]
    
    for paragraph in allnews.paragraphs:
        word_matched=0
        letter_matched=0
        for word in key_words:
            word=word.lower()
            if paragraph.title is not None and paragraph.body is not None:
                if word in paragraph.title.lower().split():
                    word_matched+=1
                    letter_matched+=len(word)
        if word_matched>=min_count and paragraph.date is not None:
            paragraph.setCounts(word_matched)
            paragraph.setLetterMatched(letter_matched)
            para_list.append(paragraph)
    return para_list

In [17]:
def sortNews(para_list):
    para_list.sort(key=lambda item:(item.cnt, item.ltr, item.date.year, item.date.month, item.date.day), reverse=True)
    
    return para_list

In [18]:
list_of_news=[]
searched_word=""
selected_item=""
total_page=0
content_per_page=5

In [19]:
app = Flask(__name__)

In [20]:
def singleNewsFromKey(searched_key):
    para_list = searchNews(searched_key, 1)
    para_list = sortNews(para_list)

    return para_list

In [21]:
def multiNewsFromKey(searched_key):
    para_list = searchNews(searched_key, len(searched_key.split()))
    para_list = sortNews(para_list)

    return para_list

In [22]:
@app.route('/show_result', methods=['GET', 'POST'])
def show_result():
    global searched_word
    searched_word = request.form['news_keyword']
    global selected_item
    selected_item = request.form.get('item')
    
    global list_of_news
        
    if selected_item == "single":
        list_of_news=singleNewsFromKey(searched_word)

        global total_page
        total_page=math.ceil(len(list_of_news)/content_per_page)

        list_ten=[]
        for i in range(min(len(list_of_news), content_per_page)):
            list_ten.append(list_of_news[i])
            
        rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word, current_page=0)
    
    else:
        list_of_news=multiNewsFromKey(searched_word)
            
        if len(list_of_news) == 0:
            rendered = render_template('ani_show_multi_news.html', multi_news="Not enough content to show. For multi document summary, all the keywords should match.")
            return rendered
        
        list_ten=[]
        for i in range(min(len(list_of_news), content_per_page)):
            list_ten.append(list_of_news[i])
            
        rendered = render_template('ani_show_multi_news.html', multi_news=getMultiDocSummaryFromList(list_ten), keyword=searched_word)
    
    return rendered

In [23]:
@app.route('/show_other_page', methods=['POST'])
def show_other_page():
    page_number = int(request.form['page_number'])
        
    list_ten=[]
    for i in range(page_number*content_per_page, min(len(list_of_news), (page_number+1)*content_per_page)):
        list_ten.append(list_of_news[i])

    rendered = render_template('ani_show_news.html', news_list=getSummarizedFromList(list_ten), number_of_pages=total_page, keyword=searched_word, current_page=page_number)

    return rendered

In [ ]:
@app.route("/")
def search_key():
    rendered = render_template('ani_search.html')
    return rendered

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('0.0.0.0', 9001, app)

 * Running on http://0.0.0.0:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Dec/2018 08:29:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Dec/2018 08:29:21] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Dec/2018 08:31:36] "GET / HTTP/1.1" 200 -
[2018-12-15 08:31:48,627] ERROR in app: Exception on /show_result [POST]
Traceback (most recent call last):
  File "/home/rabbi/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/rabbi/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/rabbi/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/rabbi/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/home/rabbi/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_disp